<a href="https://colab.research.google.com/github/viotsoft/apps/blob/main/%D0%94%D0%97_3_%D0%9A%D1%80%D0%B0%D0%B2%D1%87%D0%B5%D0%BD%D0%BA%D0%BE_%D0%A1%D0%B5%D1%80%D0%B3%D1%96%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gdown
import pickle
import numpy as np
import pandas as pd

# Завантаження моделі
url = "https://drive.google.com/uc?id=1281E0CDneuKdflWFBUvuyUzujpdGVImz"
output = "nlp_model.pkl"
gdown.download(url, output, quiet=False)

with open(output, "rb") as f:
    nlp_model = pickle.load(f)

# Створення DataFrame з тривимірними векторами
data = []
for word, vec in nlp_model.items():
    vec = np.array(vec).flatten()
    if len(vec) >= 3:
        data.append({"word": word, "x": vec[0], "y": vec[1], "z": vec[2]})
if not data:
    raise ValueError("Жоден вектор у моделі не має достатньої довжини для обрізки до 3.")
df_vectors = pd.DataFrame(data)
print("Перші 5 рядків DataFrame:")
print(df_vectors.head())

valid_words = df_vectors['word'].tolist()
print("\nСлова з 3-вимірними векторами:", valid_words)

# Функція пошуку найближчого слова
def find_nearest(word_vector, df):
    word_vector = np.array(word_vector).flatten()
    if len(word_vector) != 3:
        raise ValueError(f"Вектор для пошуку має довжину {len(word_vector)}, а не 3.")
    min_dist = float('inf')
    nearest_word = None
    for _, row in df.iterrows():
        vec = np.array([row['x'], row['y'], row['z']])
        dist = np.linalg.norm(word_vector - vec)
        if dist < min_dist:
            min_dist = dist
            nearest_word = row['word']
    if nearest_word is None:
        raise ValueError("Не знайдено слова з тривимірним вектором.")
    return nearest_word

# Тестування find_nearest із кількома прикладами
print("\nТестування пошуку найближчого слова:")
test_vectors = [
    np.array([0.1, 0.2, 0.3]),
    np.array([-0.5, 0.7, 0.1]),
    df_vectors.iloc[0][['x', 'y', 'z']].values  # Вектор першого слова
]
for i, vec in enumerate(test_vectors):
    nearest = find_nearest(vec, df_vectors)
    print(f"Тест {i+1}: Найближче слово до {vec}: {nearest}")

# Функція для векторного добутку
def find_orthogonal_word(word1, word2, df):
    vec1 = df[df['word'] == word1][['x', 'y', 'z']].values[0]
    vec2 = df[df['word'] == word2][['x', 'y', 'z']].values[0]
    cross_vector = np.cross(vec1, vec2)
    return find_nearest(cross_vector, df)

# Тестування для кількох пар слів
if len(valid_words) < 4:
    raise ValueError("Недостатньо слів для тестування кількох пар.")
word_pairs = [(valid_words[0], valid_words[1]), (valid_words[2], valid_words[3])]
print("\nТестування векторного добутку:")
for w1, w2 in word_pairs:
    orthogonal = find_orthogonal_word(w1, w2, df_vectors)
    print(f"Векторний добуток '{w1}' × '{w2}': найближче слово = {orthogonal}")

# Функція для кута між словами
def angle_between_words(word1, word2, df):
    vec1 = df[df['word'] == word1][['x', 'y', 'z']].values[0]
    vec2 = df[df['word'] == word2][['x', 'y', 'z']].values[0]
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    cos_angle = dot_product / (norm1 * norm2)
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    angle_rad = np.arccos(cos_angle)
    angle_deg = np.degrees(angle_rad)
    return angle_rad, angle_deg

# Тестування кута для кількох пар
print("\nТестування кута між словами:")
for w1, w2 in word_pairs:
    angle_rad, angle_deg = angle_between_words(w1, w2, df_vectors)
    print(f"Кут між '{w1}' і '{w2}': {angle_rad:.2f} радіан, {angle_deg:.2f} градусів")



Downloading...
From: https://drive.google.com/uc?id=1281E0CDneuKdflWFBUvuyUzujpdGVImz
To: /content/nlp_model.pkl
100%|██████████| 309k/309k [00:00<00:00, 68.7MB/s]

Перші 5 рядків DataFrame:
      word         x         y         z
0  country -0.080078  0.133789  0.143555
1     city -0.010071  0.057373  0.183594
2    China -0.073242  0.135742  0.108887
3     Iraq  0.191406  0.125000 -0.065430
4      oil -0.139648  0.062256 -0.279297

Слова з 3-вимірними векторами: ['country', 'city', 'China', 'Iraq', 'oil', 'town', 'Canada', 'London', 'England', 'Australia', 'Japan', 'Pakistan', 'Iran', 'gas', 'happy', 'Russia', 'Afghanistan', 'France', 'Germany', 'Georgia', 'Baghdad', 'village', 'Spain', 'Italy', 'Beijing', 'Jordan', 'Paris', 'Ireland', 'Turkey', 'Egypt', 'Lebanon', 'Taiwan', 'Tokyo', 'Nigeria', 'Vietnam', 'Moscow', 'Greece', 'Indonesia', 'sad', 'Syria', 'Thailand', 'Libya', 'Zimbabwe', 'Cuba', 'Ottawa', 'Tehran', 'Sudan', 'Kenya', 'Philippines', 'Sweden', 'Poland', 'Ukraine', 'Rome', 'Venezuela', 'Switzerland', 'Berlin', 'Bangladesh', 'Portugal', 'Ghana', 'Athens', 'king', 'Madrid', 'Somalia', 'Dublin', 'Qatar', 'Chile', 'Islamabad', 'Bahrain', 

Висновки:
1. Модель успішно завантажена та перетворена в DataFrame із тривимірними векторами.
2. Функція пошуку найближчого слова працює коректно, повертаючи слова з моделі на основі евклідової відстані.
3. Векторний добуток показує слова, ортогональні до площини двох векторів; результати можуть бути абстрактними, але відображають геометричну інтерпретацію.
4. Кути між словами вказують на семантичну схожість: менші кути (близько 0°) — більша схожість, більші (близько 90°) — менша схожість.